***Part 1:*** 

Building a DF for Toronto neighborhoods

***First I will import all of the libaries***
___

In [1]:
import pandas as pd
import numpy as np
import json 
!conda install -c conda-forge geopy --yes
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


**Now I will import beautifulSoup**
___

In [2]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from urllib import request, response, error, parse
from urllib.request import urlopen

**Now I will use BeautifulSoup to get the data from the wiki page**
___

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen (url)
soup = BeautifulSoup(html, "lxml")

#print(soup.prettify())     #I only commented this line after I ran it and analyzed it, so it wouldn't take up so much space

***Below I will use BeautifulSoup to find the table on the Wiki Page and create a loop to identify each of the columns and cells from the Wiki page and turn them into variables***
___

In [4]:
can_table = soup.find("table",{'class':'wikitable sortable'})
postal_codes = []
boroughs = []
neighborhoods = []

for table in can_table:
    rows = can_table.find_all('tr')
    
    for row in rows:
        cells = row.find_all('td')
        
        
        if len(cells) > 1:
            postal_code = cells[0]
            postal_codes.append(postal_code.text.strip())
            
            borough = cells[1]
            boroughs.append(borough.text.strip())
            
            neighborhood = cells[2]
            neighborhoods.append(neighborhood.text.strip())

**Below I make the Dataframe from the the variables identified in the above step**
___

In [5]:
Toronto_df = pd.DataFrame(postal_codes,
                          columns = ['Postal Code'])
Toronto_df['Borough'] = boroughs
Toronto_df['Neighborhood'] = neighborhoods
Toronto_df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
355,M5Z,Not assigned,
356,M6Z,Not assigned,
357,M7Z,Not assigned,
358,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


**Below I will replace all not assigned values with NaN, replace NaN Neighborhood's with their Borough name, drop na Boroughs, combine the Neighborhoods of the same Postal Codes (seperated by ", "), and display the first 12 rows of the DataFrame**
___

In [11]:
Toronto_df.replace("Not assigned", np.nan, inplace = True)
Toronto_df['Neighborhood'].replace(np.nan, Toronto_df['Borough'], inplace = True)
Toronto_df.dropna(subset=["Borough"], axis=0, inplace=True)
Toronto_df.groupby('Postal Code')['Neighborhood'].apply(', '.join).reset_index()
Toronto_df.reset_index(drop=True, inplace=True)
Toronto_df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


___
*saved df to pickle to use in follow-on questions*

In [8]:
Toronto_df.to_pickle('Toronoto.pkl')

**Display the DF size with the .shape method**
___

In [9]:
Toronto_df.shape

(206, 3)